Практическая работа №6

По дисциплине: Технологии машинного обучения в кибербезопасности

Выполнил студент группы: ББМО-02-23

Брестер Андрей Николаевич

In [ ]:
!pip install tensorflow keras numpy
!pip install adversarial-robustness-toolbox

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from art.estimators.classification import PyTorchClassifier
from art.attacks.evasion import FastGradientMethod
from art.defences.trainer import AdversarialTrainer

# Загрузка данных CIFAR-10 и их предобработка
def load_cifar10_data():
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Нормализация данных
    ])
    train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    return train_dataset, test_dataset

train_dataset, test_dataset = load_cifar10_data()

# Создание модели нейронной сети в PyTorch
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Обучение модели
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}')

# Оценка модели на тестовых данных
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    print(f'Accuracy on test examples: {accuracy * 100:.2f}%')

# Загрузка данных и создание загрузчиков данных
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Инициализация модели, оптимизатора и критерия
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Обучение модели
train_model(model, train_loader, criterion, optimizer)

# Оценка модели на тестовых данных
evaluate_model(model, test_loader)

# Создание ART классификатора PyTorch
classifier = PyTorchClassifier(
    model=model,
    clip_values=(0.0, 1.0),  # Нормализация входных данных
    loss=criterion,
    optimizer=optimizer,
    input_shape=(3, 32, 32),
    nb_classes=10,
)

# Получение тестовых данных и их нормализация
x_test = np.array([test_dataset[i][0].numpy() for i in range(len(test_dataset))])
y_test = np.array([test_dataset[i][1] for i in range(len(test_dataset))])
x_test = (x_test / 2) + 0.5  # Инверсная нормализация для ART

x_train = np.array([train_dataset[i][0].numpy() for i in range(len(train_dataset))])
y_train = np.array([train_dataset[i][1] for i in range(len(train_dataset))])
x_train = (x_train / 2) + 0.5  # Инверсная нормализация для ART

# Атака FGM и оценка модели на атакованных данных
attack = FastGradientMethod(estimator=classifier, eps=0.2)
x_test_adv = attack.generate(x=x_test)
predictions_adv = classifier.predict(x_test_adv)
accuracy_adv = np.sum(np.argmax(predictions_adv, axis=1) == y_test) / len(y_test)
print("Accuracy on adversarial test examples: {}%".format(accuracy_adv * 100))

# Защита от атаки и оценка модели на атакованных и оригинальных данных
trainer = AdversarialTrainer(classifier, attack)
trainer.fit(x_train, y_train)
predictions_defended = trainer.predict(x_test_adv)
accuracy_defended = np.sum(np.argmax(predictions_defended, axis=1) == y_test) / len(y_test)
print("Accuracy on defended adversarial test examples: {}%".format(accuracy_defended * 100))

predictions_original = trainer.predict(x_test)
accuracy_original = np.sum(np.argmax(predictions_original, axis=1) == y_test) / len(y_test)
print("Accuracy on defended benign test examples: {}%".format(accuracy_original * 100))

Files already downloaded and verified
Files already downloaded and verified
Epoch [1/10], Loss: 1.2927
Epoch [2/10], Loss: 0.9062
Epoch [3/10], Loss: 0.7190
Epoch [4/10], Loss: 0.5560
Epoch [5/10], Loss: 0.4033
Epoch [6/10], Loss: 0.2596
Epoch [7/10], Loss: 0.1625
Epoch [8/10], Loss: 0.1022
Epoch [9/10], Loss: 0.0873
Epoch [10/10], Loss: 0.0678
Accuracy on test examples: 72.69%
Accuracy on adversarial test examples: 20.4%


Precompute adv samples:   0%|          | 0/1 [00:00<?, ?it/s]

Adversarial training epochs:   0%|          | 0/20 [00:00<?, ?it/s]

Accuracy on defended adversarial test examples: 35.199999999999996%
Accuracy on defended benign test examples: 70.59%
